In [1]:
import os
import pickle
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model, Model
from keras_self_attention import SeqSelfAttention
from sklearn.preprocessing import OneHotEncoder



Using TensorFlow backend.


In [2]:
def main():
    """Function for removing sentiment words from a given text"""

    data = pd.read_csv('dataSampleTest.csv')
    data.columns = ["sarcasmText", "text"] # Keeping only the neccessary columns

    stop = stopwords.words('english')
    data['text_without_stopwords'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
    data['text_without_stopwords'] = data['text_without_stopwords'].apply(lambda x: x.lower())
    data['text'] = data['text'].apply(lambda x: x.lower())

    model_dir = "./models"
    token_path = os.path.join(model_dir, "token_" + "v1")
    complete_token_path = token_path + ".pickle"
    with open(complete_token_path, 'rb') as handle:
        tokenizer = pickle.load(handle)

    maxlen =30
    X = tokenizer.texts_to_sequences(data['text'].values)
    X = pad_sequences(X, maxlen=maxlen)
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
    print(X.shape)

    max_features = 20000
    enc = OneHotEncoder(handle_unknown='ignore', n_values=max_features, sparse=False)
    
    x_train_one_hot = enc.fit_transform(X)
    x_train_one_hot = np.reshape(x_train_one_hot, (X.shape[0], maxlen, max_features))

    model_name = "dummy_model_" + "v1" + "_embed_" + str(128) + "_batch_size_" + str(32)
    trained_model = os.path.join(model_dir, model_name) + "_final.hdf5"

    model = load_model(trained_model, custom_objects={'SeqSelfAttention': SeqSelfAttention})

    feat_dir = "./features"
    if not os.path.exists(feat_dir):
        os.makedirs(feat_dir)
    print(model.summary())

    dense_model = Model(inputs=model.input, outputs=model.get_layer('seq_self_attention_1').output)
    dense_feature, attn_weight = dense_model.predict(X)

    new_data_all = np.zeros((attn_weight.shape[0], attn_weight.shape[1]))
    for i in range(0, attn_weight.shape[0]):
        current_max_array = attn_weight[i].max(0)
        temp_list = []
        for k in range(0, current_max_array.shape[0]):
            if current_max_array[k] != 0:
                temp_list.append(current_max_array[k])

        current_mean = np.mean(temp_list)
        current_std = np.std(temp_list)
        num_higher = current_mean + 1*(current_std)
        num_lower = current_mean - 1.5*(current_std)
        high_outlier = (current_max_array <= num_higher).astype(int)
        low_outlier = (current_max_array > num_lower).astype(int)
        context_ones = high_outlier*low_outlier
        new_data = X[i] * context_ones
        new_data_all[i] = new_data

    def sequence_to_text(list_of_indices):
        words = [reverse_word_map.get(letter) for letter in list_of_indices]
        return words

    my_texts = list(map(sequence_to_text, new_data_all))
    correct_sent = list(data["text"])
    all_sentences = []
    new_training_output = []
    for i in range(0, len(my_texts)):
        each_new = [x for x in my_texts[i] if x is not None]
        each_new = " ".join(each_new)
        if each_new != "":
            rem = correct_sent[i]
            new_training_output.append(rem)
            all_sentences.append(each_new)
        else:
            print(i, correct_sent[i])

    print(len(all_sentences), len(new_training_output))
    for i in range(0, len(new_training_output)):
        print(all_sentences[i], " ----------- ", new_training_output[i])
main()


(205, 30)


c:\users\salil\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:326: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
c:\users\salil\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 128)           2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 200)           263200    
_________________________________________________________________
seq_self_attention_1 (SeqSel [(None, 30, 200), (None,  12865     
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               210600    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 302       
Total params: 3,046,967
Trainable params: 2,836,367
Non-trainable params: 210,600
___________________________________________